# Creating The URL Of The Filing and Fetching the Filing Data With the Help of URL

In [1]:
#importing libraries
import pandas as pd
import csv
import urllib.request

In [2]:
df =pd.read_csv(r"C:\Users\souro\OneDrive\Desktop\BlackCoffer_project\cik_list.csv", delimiter = ',')

In [3]:
#adding 'https://www.sec.gov/Archives/ to every cells of column F (cik_list.xlsx) to access link to the financial report. '
df['SECFNAME'] ='https://www.sec.gov/Archives/' +df['SECFNAME'].astype(str)

In [4]:
#saving the dataframe to another file
df.to_csv("cik_list1.csv")

In [5]:
#filing downloadable link file
filing_link =r'C:\Users\souro\OneDrive\Desktop\Blackcoffer\clicklist1.csv'

In [6]:
#downlaoding filing file from the web
cikListFile = pd.read_csv(filing_link)
for index, row in cikListFile.iterrows():
    processingFile=row['SECFNAME'].split('/')
    url =row['SECFNAME']
    inputFile = processingFile[7]
    name = str(inputFile)
    urllib.request.urlretrieve(url, 'C:/Users/souro/OneDrive/Desktop/BlackCoffer_project/Filing_Details/'+name) 

# Extracting and Analyzing the Data

In [9]:
#importing library
import os
import re
import pandas as pd
from nltk.tokenize import RegexpTokenizer, sent_tokenize
import numpy as np

In [10]:
# Text extraction patterns
mda_regex = r"item[^a-zA-Z\n]*\d\s*\.\s*management\'s discussion and analysis.*?^\s*item[^a-zA-Z\n]*\d\s*\.*"
qqd_regex = r"item[^a-zA-Z\n]*\d[a-z]?\.?\s*Quantitative and Qualitative Disclosures about " \
            r"Market Risk.*?^\s*item\s*\d\s*"
riskfactor_regex = r"item[^a-zA-Z\n]*\d[a-z]?\.?\s*Risk Factors.*?^\s*item\s*\d\s*"

In [11]:
# Filepath locations of all files
stopWordsFile = r'C:\Users\souro\OneDrive\Desktop\BlackCoffer_project\StopWords_Generic.txt'
positiveWordsFile = r'C:\Users\souro\OneDrive\Desktop\BlackCoffer_project\PositiveWords.txt'
nagitiveWordsFile = r'C:\Users\souro\OneDrive\Desktop\BlackCoffer_project\NegativeWords.txt'
uncertainty_dictionaryFile = r'C:\Users\souro\OneDrive\Desktop\BlackCoffer_project\uncertainty_dictionary.txt'
constraining_dictionaryFile = r'C:\Users\souro\OneDrive\Desktop\BlackCoffer_project\constraining_dictionary.txt'

In [12]:
#function for extracting data
def rawdata_extract(path, cikListFile):
    html_regex = re.compile(r'<.*?>')
    extraxted_data=[]
    
    
    cikListFile = pd.read_csv(cikListFile)
    for index, row in cikListFile.iterrows():
        processingFile=row['SECFNAME'].split('/')
        inputFile = processingFile[3]
        cik=row['CIK']
        coname=row['CONAME']
        fyrmo=row['FYRMO']
        fdate = row['FDATE']
        form = row['FORM']
        secfname=row['SECFNAME']
        for fileName in os.listdir(path):
            filenameopen = os.path.join(path, fileName)
            dirFileName = filenameopen.split('\\')
            currentFile=dirFileName[7]

            if os.path.isfile(filenameopen) and currentFile == inputFile :
                resultdict = dict()
                resultdict['CIK'] = cik
                resultdict['CONAME'] = coname
                resultdict['FYRMO'] = fyrmo
                resultdict['FDATE'] = fdate
                resultdict['FORM'] = form
                resultdict['SECFNAME'] = secfname
                
                with open(filenameopen, 'r', encoding='utf-8', errors="replace") as in_file:
                    content = in_file.read()
                    content = re.sub(html_regex,'',content)
                    content = content.replace('&nbsp;','')
                    content = re.sub(r'&#\d+;', '', content)
                    matches_mda = re.findall(mda_regex, content, re.IGNORECASE | re.DOTALL | re.MULTILINE)
                    if matches_mda:
                        result = max(matches_mda, key=len)
                        result = str(result).replace('\n', '')
                        resultdict['mda_extract'] = result
                    else:
                        resultdict['mda_extract'] = ""
                    match_qqd = re.findall(qqd_regex, content, re.IGNORECASE | re.DOTALL | re.MULTILINE)
                    if match_qqd:
                        result_qqd = max(match_qqd, key=len)
                        result_qqd = str(result_qqd).replace('\n','')
                        resultdict['qqd_extract']= result_qqd
                    else:
                        resultdict['qqd_extract'] = ""
                    match_riskfactor = re.findall(riskfactor_regex, content, re.IGNORECASE | re.DOTALL | re.MULTILINE)
                    if match_riskfactor:
                        result_riskfactor = max(match_riskfactor, key=len)
                        result_riskfactor = str(result_riskfactor).replace('\n', '')
                        resultdict['riskfactor_extract'] = result_riskfactor
                    else:
                        resultdict['riskfactor_extract'] = ""
                    extraxted_data.append(resultdict)

                in_file.close()

    return extraxted_data

# 1.1 Cleaning Using Stop Words(The stop word from the Stop Words dictionary Will pass to the Tokenizer to clean the Stop Words From The Text)

### Dictionary of Stop words

In [13]:
#Loading stop words dictionary
with open(stopWordsFile ,'r') as stop_words:
    stopWords = stop_words.read().lower()
stopWordList = stopWords.split('\n')
print(stopWordList)
stopWordList[-1:] = []

['about', 'above', 'after', 'again', 'all', 'am', 'among', 'an', 'and', 'any', 'are', 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'did', 'do', 'does', 'doing', 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'has', 'have', 'having', 'he', 'her', 'here', 'hers', 'herself', 'him', 'himself', 'his', 'how', 'if', 'in', 'into', 'is', 'it', 'its', 'itself', 'just', 'me', 'more', 'most', 'my', 'myself', 'no', 'nor', 'not', 'now', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 'same', 'she', 'should', 'so', 'some', 'such', 'than', 'that', 'the', 'their', 'theirs', 'them', 'themselves', 'then', 'there', 'these', 'they', 'this', 'those', 'through', 'to', 'too', 'under', 'until', 'up', 'very', 'was', 'we', 'were', 'what', 'when', 'where', 'which', 'while', 'who', 'whom', 'why', 'with', 'you', 'your', 'yours', 'yourself', 'yourselves']


## Creating Tokenizer for Tokenize the text into word or coverting the text into a list of tokens

In [14]:
# Tokenizer
def tokenizer(text):
    text = text.lower()
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(text)
    filtered_words = list(filter(lambda token: token not in stopWordList, tokens))
    return filtered_words

## Total words after Cleaning

In [17]:
#Function for counting total words
def total_word_count(text):
    tokens = tokenizer(text)
    return len(tokens)

# 1.2 Creating Dictionary For Negative And Postitive Words

### Dictionary for Postive Word

In [56]:
# Loading positive words
with open(positiveWordsFile,'r') as posfile:
    positivewords=posfile.read().lower()
positiveWordList=positivewords.split('\n')
print(positiveWordList)

['able', 'abundance', 'abundant', 'acclaimed', 'accomplish', 'accomplished', 'accomplishes', 'accomplishing', 'accomplishment', 'accomplishments', 'achieve', 'achieved', 'achievement', 'achievements', 'achieves', 'achieving', 'adequately', 'advancement', 'advancements', 'advances', 'advancing', 'advantage', 'advantaged', 'advantageous', 'advantageously', 'advantages', 'alliance', 'alliances', 'assure', 'assured', 'assures', 'assuring', 'attain', 'attained', 'attaining', 'attainment', 'attainments', 'attains', 'attractive', 'attractiveness', 'beautiful', 'beautifully', 'beneficial', 'beneficially', 'benefit', 'benefited', 'benefiting', 'benefitted', 'benefitting', 'best', 'better', 'bolstered', 'bolstering', 'bolsters', 'boom', 'booming', 'boost', 'boosted', 'breakthrough', 'breakthroughs', 'brilliant', 'charitable', 'collaborate', 'collaborated', 'collaborates', 'collaborating', 'collaboration', 'collaborations', 'collaborative', 'collaborator', 'collaborators', 'compliment', 'complime

### Dictionary For Negative Word

In [58]:
# Loading negative words
with open(nagitiveWordsFile ,'r') as negfile:
    negativeword=negfile.read().lower()
negativeWordList=negativeword.split('\n')
print(negativeWordList)

['absence', 'absences', 'absenteeism', 'abuse', 'abused', 'abuses', 'abusing', 'abusive', 'abusively', 'abusiveness', 'accident', 'accidental', 'accidentally', 'accidents', 'accusation', 'accusations', 'accuse', 'accused', 'accuses', 'accusing', 'acquiesce', 'acquiesced', 'acquiesces', 'acquiescing', 'acquit', 'acquits', 'acquittal', 'acquittals', 'acquitted', 'acquitting', 'adulterate', 'adulterated', 'adulterating', 'adulteration', 'adulterations', 'adversarial', 'adversaries', 'adversary', 'adverse', 'adversely', 'adversities', 'adversity', 'aftermath', 'aftermaths', 'against', 'aggravate', 'aggravated', 'aggravates', 'aggravating', 'aggravation', 'aggravations', 'alerted', 'alerting', 'alienate', 'alienated', 'alienates', 'alienating', 'alienation', 'alienations', 'allegation', 'allegations', 'allege', 'alleged', 'allegedly', 'alleges', 'alleging', 'annoy', 'annoyance', 'annoyances', 'annoyed', 'annoying', 'annoys', 'annul', 'annulled', 'annulling', 'annulment', 'annulments', 'annu

# 1.3 Extracting Derived Variables

### Calculating Positive Score

In [55]:
#function to calculate positive score 
def positive_score(text):
    numPosWords = 0
    rawToken = tokenizer(text)
    for word in rawToken:
        if word in positiveWordList:
            numPosWords  += 1
    
    sumPos = numPosWords
    return sumPos

### Calculating  Negative Score

In [3]:
#function ot calculate negative score
def negative_word(text):
    numNegWords=0
    rawToken = tokenizer(text)
    for word in rawToken:
        if word in negativeWordList:
            numNegWords -=1
    sumNeg = numNegWords 
    sumNeg = sumNeg * -1
    return sumNeg

### Calculating Polarity Score

In [ ]:
#function for calculating polarity score
def polarity_score(positiveScore, negativeScore):
    pol_score = (positiveScore - negativeScore) / ((positiveScore + negativeScore) + 0.000001)
    return pol_score

### Calculating Subjectivity score

In [32]:
#function for calculating polarity score
def subjectivity_score(positiveScore, negativeScore, text):
    subj_score = (positiveScore - negativeScore) / total_word_count(text)
    return subj_score

# 2 Analysis of Readability

### Calculating Average Sentence Per Length

In [33]:
#function for caluculating average sentence length
def average_sentence_length(text):
    sentence_list = sent_tokenize(text)
    tokens = tokenizer(text)
    totalWordCount = len(tokens)
    totalSentences = len(sentence_list)
    average_sent = 0
    if totalSentences != 0:
        average_sent = totalWordCount / totalSentences
    
    average_sent_length= average_sent
    
    return round(average_sent_length)

### Calculating percentage of Complex Word

In [34]:
#function for calculating percentage of complex word 
def percentage_complex_word(text):
    tokens = tokenizer(text)
    complexWord = 0
    complex_word_percentage = 0
    
    for word in tokens:
        vowels=0
        if word.endswith(('es','ed')):
            pass
        else:
            for w in word:
                if(w=='a' or w=='e' or w=='i' or w=='o' or w=='u'):
                    vowels += 1
            if(vowels > 2):
                complexWord += 1
    if len(tokens) != 0:
        complex_word_percentage = complexWord/len(tokens)
    
    return complex_word_percentage

### Calculating FOG Index

In [35]:
#function for calculaitng fog index
def fog_index(averageSentenceLength, percentageComplexWord):
    fogIndex = 0.4 * (averageSentenceLength + percentageComplexWord)
    return fogIndex

# 3	Average Number of Words Per Sentence

### Calculating average number of words per sentence

In [39]:
#function for calculating average number of words
def average_word_length(text):
    sentence_list = sent_tokenize(text)
    tokens = tokenizer(text)
    totalWordCount = len(tokens)
    totalSentences = len(sentence_list)
    average_word = 0
    if totalSentences != 0:
        average_word = totalWordCount / totalSentences
    
    average_word_length= average_word
    
    return round(average_word_length)

# Calculating Percentage of Complex word

In [40]:
#Function for calculating percentage of Complex word
def complex_word_count(text):
    tokens = tokenizer(text)
    complexWord = 0
    
    for word in tokens:
        vowels=0
        if word.endswith(('es','ed')):
            pass
        else:
            for w in word:
                if(w=='a' or w=='e' or w=='i' or w=='o' or w=='u'):
                    vowels += 1
            if(vowels > 2):
                complexWord += 1
    return complexWord

# 4	Complex Word Count

### Calculating Complex Words

In [37]:
#function for couting complex words
def complex_word_count(text):
    tokens = tokenizer(text)
    complexWord = 0
    
    for word in tokens:
        vowels=0
        if word.endswith(('es','ed')):
            pass
        else:
            for w in word:
                if(w=='a' or w=='e' or w=='i' or w=='o' or w=='u'):
                    vowels += 1
            if(vowels > 2):
                complexWord += 1
    return complexWord

# Word Count

In [38]:
#Function for counting total words
def total_word_count(text):
    tokens = tokenizer(text)
    return len(tokens)

## Loading Uncertanity File

In [42]:
with open(uncertainty_dictionaryFile ,'r') as uncertain_dict:
    uncertainDict=uncertain_dict.read().lower()
uncertainDictionary = uncertainDict.split('\n')
print(uncertainDictionary)

['abeyance', 'abeyances', 'almost', 'alteration', 'alterations', 'ambiguities', 'ambiguity', 'ambiguous', 'anomalies', 'anomalous', 'anomalously', 'anomaly', 'anticipate', 'anticipated', 'anticipates', 'anticipating', 'anticipation', 'anticipations', 'apparent', 'apparently', 'appear', 'appeared', 'appearing', 'appears', 'approximate', 'approximated', 'approximately', 'approximates', 'approximating', 'approximation', 'approximations', 'arbitrarily', 'arbitrariness', 'arbitrary', 'assume', 'assumed', 'assumes', 'assuming', 'assumption', 'assumptions', 'believe', 'believed', 'believes', 'believing', 'cautious', 'cautiously', 'cautiousness', 'clarification', 'clarifications', 'conceivable', 'conceivably', 'conditional', 'conditionally', 'confuses', 'confusing', 'confusingly', 'confusion', 'contingencies', 'contingency', 'contingent', 'contingently', 'contingents', 'could', 'crossroad', 'crossroads', 'depend', 'depended', 'dependence', 'dependencies', 'dependency', 'dependent', 'depending'

### Calculating Uncertainity Score

In [43]:
#function for calculating uncertainity score
def uncertainty_score(text):
    uncertainWordnum =0
    rawToken = tokenizer(text)
    for word in rawToken:
        if word in uncertainDictionary:
            uncertainWordnum +=1
    sumUncertainityScore = uncertainWordnum 
    
    return sumUncertainityScore

## Loading Constraining File

In [44]:
with open(constraining_dictionaryFile ,'r') as constraining_dict:
    constrainDict=constraining_dict.read().lower()
constrainDictionary = constrainDict.split('\n')
print(constrainDictionary)

['abide', 'abiding', 'bound', 'bounded', 'commit', 'commitment', 'commitments', 'commits', 'committed', 'committing', 'compel', 'compelled', 'compelling', 'compels', 'comply', 'compulsion', 'compulsory', 'confine', 'confined', 'confinement', 'confines', 'confining', 'constrain', 'constrained', 'constraining', 'constrains', 'constraint', 'constraints', 'covenant', 'covenanted', 'covenanting', 'covenants', 'depend', 'dependance', 'dependances', 'dependant', 'dependencies', 'dependent', 'depending', 'depends', 'dictate', 'dictated', 'dictates', 'dictating', 'directive', 'directives', 'earmark', 'earmarked', 'earmarking', 'earmarks', 'encumber', 'encumbered', 'encumbering', 'encumbers', 'encumbrance', 'encumbrances', 'entail', 'entailed', 'entailing', 'entails', 'entrench', 'entrenched', 'escrow', 'escrowed', 'escrows', 'forbade', 'forbid', 'forbidden', 'forbidding', 'forbids', 'impair', 'impaired', 'impairing', 'impairment', 'impairments', 'impairs', 'impose', 'imposed', 'imposes', 'impos

### Calculating Constraining Score

In [45]:
#function for calculating Constraining score
def constraining_score(text):
    constrainWordnum =0
    rawToken = tokenizer(text)
    for word in rawToken:
        if word in constrainDictionary:
            constrainWordnum +=1
    sumConstrainScore = constrainWordnum 
    
    return sumConstrainScore

## Calculating positive word proportion

In [46]:
#function for positive word proportion
def positive_word_prop(positiveScore,wordcount):
    positive_word_proportion = 0
    if wordcount !=0:
        positive_word_proportion = positiveScore / wordcount
        
    return positive_word_proportion

## Calculating negative word proportion

In [47]:
#function for negative word proportion 
def negative_word_prop(negativeScore,wordcount):
    negative_word_proportion = 0
    if wordcount !=0:
        negative_word_proportion = negativeScore / wordcount
        
    return negative_word_proportion

## Calculating Uncertain Word Proportion

In [60]:
#function for uncertain word proportion
def uncertain_word_prop(uncertainScore,wordcount):
    uncertain_word_proportion = 0
    if wordcount !=0:
        uncertain_word_proportion = uncertainScore / wordcount
        
    return uncertain_word_proportion

## Calculating Constraining Word Proportion

In [61]:
#function for Constraining Word Proportion
def constraining_word_prop(constrainingScore,wordcount):
    constraining_word_proportion = 0
    if wordcount !=0:
        constraining_word_proportion = constrainingScore / wordcount
        
    return constraining_word_proportion

## Calculating Constraining words Over the Whole report

In [62]:
#function for Constraining words Over the Whole report
def constrain_word_whole(mdaText,qqdmrText,rfText):
    wholeDoc = mdaText + qqdmrText + rfText
    constrainWordnumWhole =0
    rawToken = tokenizer(wholeDoc)
    for word in rawToken:
        if word in constrainDictionary:
            constrainWordnumWhole +=1
    sumConstrainScoreWhole = constrainWordnumWhole 
    
    return sumConstrainScoreWhole

## Input to the function

In [52]:
inputDirectory = r'C:\Users\souro\OneDrive\Desktop\BlackCoffer_project\Filing_Details'
masterFile = r'C:\Users\souro\OneDrive\Desktop\BlackCoffer_project\cik_list.csv'
dataList = rawdata_extract( inputDirectory , masterFile )
df = pd.DataFrame(dataList)

In [53]:
df

,CIK,CONAME,FYRMO,FDATE,FORM,SECFNAME,mda_extract,qqd_extract,riskfactor_extract
0,3662,SUNBEAM CORP/FL/,199803,3/6/1998,10-K405,edgar/data/3662/0000950170-98-000413.txt,ITEM 7. MANAGEMENT'S DISCUSSION AND ANALYSIS O...,,
1,3662,SUNBEAM CORP/FL/,199805,5/15/1998,10-Q,edgar/data/3662/0000950170-98-001001.txt,ITEM 2. MANAGEMENT'S DISCUSSION AND ANALYSIS O...,,
2,3662,SUNBEAM CORP/FL/,199808,8/13/1998,NT 10-Q,edgar/data/3662/0000950172-98-000783.txt,,,
3,3662,SUNBEAM CORP/FL/,199811,11/12/1998,10-K/A,edgar/data/3662/0000950170-98-002145.txt,ITEM 7. MANAGEMENT'S DISCUSSION AND ANALYSIS O...,,
4,3662,SUNBEAM CORP/FL/,199811,11/16/1998,NT 10-Q,edgar/data/3662/0000950172-98-001203.txt,,,
...,...,...,...,...,...,...,...,...,...
147,12239,SPHERIX INC,200704,4/2/2007,10-K,edgar/data/12239/0001104659-07-024804.txt,,,
148,12239,SPHERIX INC,200705,5/16/2007,NT 10-Q,edgar/data/12239/0001104659-07-040463.txt,,,
149,12239,SPHERIX INC,200705,5/18/2007,10-Q,edgar/data/12239/0001104659-07-041441.txt,,,Item 1a. Risk FactorsInaddition to the other i...
150,12239,SPHERIX INC,200705,5/23/2007,10-K/A,edgar/data/12239/0001104659-07-042333.txt,,,Item 1A. RISK FACTORSAny of the riskfactors we...


## Adding  Output Variables to the DataFrame

In [63]:
df['mda_positive_score'] = df.mda_extract.apply(positive_score)
df['mda_negative_score'] = df.mda_extract.apply(negative_word)
df['mda_polarity_score'] = np.vectorize(polarity_score)(df['mda_positive_score'],df['mda_negative_score'])
df['mda_average_sentence_length'] = df.mda_extract.apply(average_sentence_length)
df['mda_percentage_of_complex_words'] = df.mda_extract.apply(percentage_complex_word)
df['mda_fog_index'] = np.vectorize(fog_index)(df['mda_average_sentence_length'],df['mda_percentage_of_complex_words'])
df['mda_complex_word_count']= df.mda_extract.apply(complex_word_count)
df['mda_word_count'] = df.mda_extract.apply(total_word_count)
df['mda_uncertainty_score']=df.mda_extract.apply(uncertainty_score)
df['mda_constraining_score'] = df.mda_extract.apply(constraining_score)
df['mda_positive_word_proportion'] = np.vectorize(positive_word_prop)(df['mda_positive_score'],df['mda_word_count'])
df['mda_negative_word_proportion'] = np.vectorize(negative_word_prop)(df['mda_negative_score'],df['mda_word_count'])
df['mda_uncertainty_word_proportion'] = np.vectorize(uncertain_word_prop)(df['mda_uncertainty_score'],df['mda_word_count'])
df['mda_constraining_word_proportion'] = np.vectorize(constraining_word_prop)(df['mda_constraining_score'],df['mda_word_count'])

df['qqdmr_positive_score'] = df.qqd_extract.apply(positive_score)
df['qqdmr_negative_score'] = df.qqd_extract.apply(negative_word)
df['qqdmr_polarity_score'] = np.vectorize(polarity_score)(df['qqdmr_positive_score'],df['qqdmr_negative_score'])
df['qqdmr_average_sentence_length'] = df.qqd_extract.apply(average_sentence_length)
df['qqdmr_percentage_of_complex_words'] = df.qqd_extract.apply(percentage_complex_word)
df['qqdmr_fog_index'] = np.vectorize(fog_index)(df['qqdmr_average_sentence_length'],df['qqdmr_percentage_of_complex_words'])
df['qqdmr_complex_word_count']= df.qqd_extract.apply(complex_word_count)
df['qqdmr_word_count'] = df.qqd_extract.apply(total_word_count)
df['qqdmr_uncertainty_score']=df.qqd_extract.apply(uncertainty_score)
df['qqdmr_constraining_score'] = df.qqd_extract.apply(constraining_score)
df['qqdmr_positive_word_proportion'] = np.vectorize(positive_word_prop)(df['qqdmr_positive_score'],df['qqdmr_word_count'])
df['qqdmr_negative_word_proportion'] = np.vectorize(negative_word_prop)(df['qqdmr_negative_score'],df['qqdmr_word_count'])
df['qqdmr_uncertainty_word_proportion'] = np.vectorize(uncertain_word_prop)(df['qqdmr_uncertainty_score'],df['qqdmr_word_count'])
df['qqdmr_constraining_word_proportion'] = np.vectorize(constraining_word_prop)(df['qqdmr_constraining_score'],df['qqdmr_word_count'])

df['rf_positive_score'] = df.riskfactor_extract.apply(positive_score)
df['rf_negative_score'] = df.riskfactor_extract.apply(negative_word)
df['rf_polarity_score'] = np.vectorize(polarity_score)(df['rf_positive_score'],df['rf_negative_score'])
df['rf_average_sentence_length'] = df.riskfactor_extract.apply(average_sentence_length)
df['rf_percentage_of_complex_words'] = df.riskfactor_extract.apply(percentage_complex_word)
df['rf_fog_index'] = np.vectorize(fog_index)(df['rf_average_sentence_length'],df['rf_percentage_of_complex_words'])
df['rf_complex_word_count']= df.riskfactor_extract.apply(complex_word_count)
df['rf_word_count'] = df.riskfactor_extract.apply(total_word_count)
df['rf_uncertainty_score']=df.riskfactor_extract.apply(uncertainty_score)
df['rf_constraining_score'] = df.riskfactor_extract.apply(constraining_score)
df['rf_positive_word_proportion'] = np.vectorize(positive_word_prop)(df['rf_positive_score'],df['rf_word_count'])
df['rf_negative_word_proportion'] = np.vectorize(negative_word_prop)(df['rf_negative_score'],df['rf_word_count'])
df['rf_uncertainty_word_proportion'] = np.vectorize(uncertain_word_prop)(df['rf_uncertainty_score'],df['rf_word_count'])
df['rf_constraining_word_proportion'] = np.vectorize(constraining_word_prop)(df['rf_constraining_score'],df['rf_word_count'])
df['constraining_words_whole_report'] = np.vectorize(constrain_word_whole)(df['mda_extract'],df['qqd_extract'],df['riskfactor_extract'])

In [64]:
df.head()

,CIK,CONAME,FYRMO,FDATE,FORM,SECFNAME,mda_extract,qqd_extract,riskfactor_extract,mda_positive_score,...,rf_fog_index,rf_complex_word_count,rf_word_count,rf_uncertainty_score,rf_constraining_score,rf_positive_word_proportion,rf_negative_word_proportion,rf_uncertainty_word_proportion,rf_constraining_word_proportion,constraining_words_whole_report
0,3662,SUNBEAM CORP/FL/,199803,3/6/1998,10-K405,edgar/data/3662/0000950170-98-000413.txt,ITEM 7. MANAGEMENT'S DISCUSSION AND ANALYSIS O...,,,17,...,0.0,0,0,0,0,0,0,0,0,10
1,3662,SUNBEAM CORP/FL/,199805,5/15/1998,10-Q,edgar/data/3662/0000950170-98-001001.txt,ITEM 2. MANAGEMENT'S DISCUSSION AND ANALYSIS O...,,,9,...,0.0,0,0,0,0,0,0,0,0,2
2,3662,SUNBEAM CORP/FL/,199808,8/13/1998,NT 10-Q,edgar/data/3662/0000950172-98-000783.txt,,,,0,...,0.0,0,0,0,0,0,0,0,0,0
3,3662,SUNBEAM CORP/FL/,199811,11/12/1998,10-K/A,edgar/data/3662/0000950170-98-002145.txt,ITEM 7. MANAGEMENT'S DISCUSSION AND ANALYSIS O...,,,41,...,0.0,0,0,0,0,0,0,0,0,39
4,3662,SUNBEAM CORP/FL/,199811,11/16/1998,NT 10-Q,edgar/data/3662/0000950172-98-001203.txt,,,,0,...,0.0,0,0,0,0,0,0,0,0,0


In [73]:
p=df

In [74]:
p.head()

,CIK,CONAME,FYRMO,FDATE,FORM,SECFNAME,mda_extract,qqd_extract,riskfactor_extract,mda_positive_score,...,rf_fog_index,rf_complex_word_count,rf_word_count,rf_uncertainty_score,rf_constraining_score,rf_positive_word_proportion,rf_negative_word_proportion,rf_uncertainty_word_proportion,rf_constraining_word_proportion,constraining_words_whole_report
0,3662,SUNBEAM CORP/FL/,199803,3/6/1998,10-K405,edgar/data/3662/0000950170-98-000413.txt,ITEM 7. MANAGEMENT'S DISCUSSION AND ANALYSIS O...,,,17,...,0.0,0,0,0,0,0,0,0,0,10
1,3662,SUNBEAM CORP/FL/,199805,5/15/1998,10-Q,edgar/data/3662/0000950170-98-001001.txt,ITEM 2. MANAGEMENT'S DISCUSSION AND ANALYSIS O...,,,9,...,0.0,0,0,0,0,0,0,0,0,2
2,3662,SUNBEAM CORP/FL/,199808,8/13/1998,NT 10-Q,edgar/data/3662/0000950172-98-000783.txt,,,,0,...,0.0,0,0,0,0,0,0,0,0,0
3,3662,SUNBEAM CORP/FL/,199811,11/12/1998,10-K/A,edgar/data/3662/0000950170-98-002145.txt,ITEM 7. MANAGEMENT'S DISCUSSION AND ANALYSIS O...,,,41,...,0.0,0,0,0,0,0,0,0,0,39
4,3662,SUNBEAM CORP/FL/,199811,11/16/1998,NT 10-Q,edgar/data/3662/0000950172-98-001203.txt,,,,0,...,0.0,0,0,0,0,0,0,0,0,0


In [76]:
inputTextCol = ['mda_extract','qqd_extract','riskfactor_extract']
FinalOutput = p.drop(inputTextCol, axis =1)

FinalOutput.head(20)

,CIK,CONAME,FYRMO,FDATE,FORM,SECFNAME,mda_positive_score,mda_negative_score,mda_polarity_score,mda_average_sentence_length,...,rf_fog_index,rf_complex_word_count,rf_word_count,rf_uncertainty_score,rf_constraining_score,rf_positive_word_proportion,rf_negative_word_proportion,rf_uncertainty_word_proportion,rf_constraining_word_proportion,constraining_words_whole_report
0,3662,SUNBEAM CORP/FL/,199803,3/6/1998,10-K405,edgar/data/3662/0000950170-98-000413.txt,17,61,-0.564103,24,...,0.0,0,0,0,0,0,0,0,0,10
1,3662,SUNBEAM CORP/FL/,199805,5/15/1998,10-Q,edgar/data/3662/0000950170-98-001001.txt,9,46,-0.672727,30,...,0.0,0,0,0,0,0,0,0,0,2
2,3662,SUNBEAM CORP/FL/,199808,8/13/1998,NT 10-Q,edgar/data/3662/0000950172-98-000783.txt,0,0,0.000000,0,...,0.0,0,0,0,0,0,0,0,0,0
3,3662,SUNBEAM CORP/FL/,199811,11/12/1998,10-K/A,edgar/data/3662/0000950170-98-002145.txt,41,119,-0.487500,23,...,0.0,0,0,0,0,0,0,0,0,39
4,3662,SUNBEAM CORP/FL/,199811,11/16/1998,NT 10-Q,edgar/data/3662/0000950172-98-001203.txt,0,0,0.000000,0,...,0.0,0,0,0,0,0,0,0,0,0
5,3662,SUNBEAM CORP/FL/,199811,11/25/1998,10-Q/A,edgar/data/3662/0000950170-98-002278.txt,19,63,-0.536585,23,...,0.0,0,0,0,0,0,0,0,0,23
6,3662,SUNBEAM CORP/FL/,199812,12/22/1998,10-Q,edgar/data/3662/0000950170-98-002401.txt,40,106,-0.452055,22,...,0.0,0,0,0,0,0,0,0,0,34
7,3662,SUNBEAM CORP/FL/,199812,12/22/1998,10-Q,edgar/data/3662/0000950170-98-002402.txt,38,102,-0.457143,22,...,0.0,0,0,0,0,0,0,0,0,32
8,3662,SUNBEAM CORP/FL/,199903,3/31/1999,NT 10-K,edgar/data/3662/0000950172-99-000362.txt,0,0,0.000000,0,...,0.0,0,0,0,0,0,0,0,0,0
9,3662,SUNBEAM CORP/FL/,199905,5/11/1999,10-K,edgar/data/3662/0000950170-99-000775.txt,71,270,-0.583578,23,...,0.0,0,0,0,0,0,0,0,0,74


In [80]:
FinalOutput.to_csv(r'C:\Users\souro\OneDrive\Desktop\BlackCoffer_project\EdgarDatatextAnalysisOutput.csv', sep=',', encoding='utf-8')